Wczytywanie zbioru danych.

In [1]:
import numpy as np
import pandas as pd

# df = pd.read_csv("reviews_bow.csv")
df = pd.read_csv("reviews_w2v.csv")
df.head()

,helpfulN,helpfulP,reviewerID,reviewerName,asin,score,unixReviewTime,0,1,2,...,590,591,592,593,594,595,596,597,598,599
0,7,6,30124,43149,2747,1.0,1333843200,0.074479,0.032809,0.016875,...,0.078766,0.047852,-0.055908,-0.064941,0.042480,0.055908,-0.000977,-0.074951,0.006104,0.071777
1,6,0,4713,31532,5394,1.0,1388188800,0.069027,0.052856,0.085506,...,0.273438,0.024780,-0.099609,-0.044678,0.251953,0.133789,-0.026001,-0.166992,-0.040283,-0.072266
2,4,2,59783,46230,7167,1.0,1356048000,0.085327,0.011286,0.024109,...,0.068848,0.086426,-0.061768,-0.038330,-0.174805,-0.253906,0.086914,-0.024048,-0.026978,0.125000
3,4,2,67587,41970,4036,1.0,1334188800,0.064137,-0.000476,-0.028126,...,-0.018921,0.394531,-0.219727,-0.018921,-0.200195,-0.037109,0.243164,-0.135742,-0.032227,-0.072754
4,1,0,65465,54619,4625,1.0,1387670400,-0.035083,0.002816,0.037427,...,0.171875,0.162109,-0.135742,0.212891,-0.052979,-0.194336,0.156250,-0.208008,0.079102,0.137695


Wybór klasyfikatora i poszukiwanie optymalnych parametrów.

In [2]:
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import f1_score

y = df["score"]
X = df[list(set(df.columns) - {"score"})]
# print("X columns:", list(X.columns), "\ny columns:", y.name)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

print("\n---Tuning hyperparameters---")

param_grid = {
    'n_estimators': [200, 350, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth': [10, 20, 30, 50, 70],
    'n_jobs': [-1],
    'random_state': [23]
}

gs = GridSearchCV(RandomForestClassifier(), param_grid, n_jobs=-1, verbose=3, scoring='f1_weighted')
gs.fit(X_train, y_train)
print(gs.best_params_)


---Tuning hyperparameters---
Fitting 5 folds for each of 10 candidates, totalling 50 fits
{'max_depth': 20, 'max_features': 'auto', 'n_estimators': 350, 'n_jobs': -1, 'random_state': 23}


Rezultaty klasyfikacji i porównania z algorytmem większościowym oraz losowym.

In [3]:
print("\n---Training classifier with best found parameters---")
clf = RandomForestClassifier(**gs.best_params_)
clf.fit(X_train, y_train)


---Training classifier with best found parameters---


RandomForestClassifier(max_depth=20, n_estimators=350, n_jobs=-1,
                       random_state=23)

Ocena klasyfikatora oraz porównanie go do algorytmów losowego oraz większościowego.

In [4]:
import random
from sklearn.metrics import f1_score, precision_score, recall_score


def scores(y_true, y_pred, name=None, df_compare=pd.DataFrame(np.zeros((5, 3)))):
    measures = ["F1", "Precision", "Recall"]
    df_scores = pd.DataFrame(columns=measures, index=list_of_labels)

    df_scores["F1"] = f1_score(y_true, y_pred, average=None, pos_label=None, labels=list_of_labels)
    df_scores["Precision"] = precision_score(y_true, y_pred, average=None, pos_label=None, labels=list_of_labels,
                                             zero_division=0)
    df_scores["Recall"] = recall_score(y_true, y_pred, average=None, pos_label=None, labels=list_of_labels)

    print(f"=================== Results: {name} ===================")
    # print(pd.DataFrame(df_scores.to_numpy() - df_compare.to_numpy()).T)
    print(df_scores.T)
    # return df_scores


list_of_labels = y_train.unique()
majority_label = y_train.value_counts().index[0]
random.seed(23)

y_pred_random, y_pred_majority = [], []

for _ in range(len(X_test)):
    y_pred_random.append(random.choice(list_of_labels))
    y_pred_majority.append(majority_label)

scores(y_test, y_pred_random, name="RandomClassifier")
scores(y_test, y_pred_majority, name="MajorityClassifier")
scores(y_test, clf.predict(X_test), name="RandomForestClassifier")
print("======================================")

=================== Results: RandomClassifier ===================
                1.0       4.0       2.0       5.0       3.0
F1         0.205526  0.202981  0.192125  0.196755  0.199559
Precision  0.204462  0.203305  0.192186  0.197378  0.199586
Recall     0.206601  0.202657  0.192064  0.196137  0.199532
=================== Results: MajorityClassifier ===================
           1.0  4.0      2.0  5.0  3.0
F1         0.0  0.0  0.33097  0.0  0.0
Precision  0.0  0.0  0.19830  0.0  0.0
Recall     0.0  0.0  1.00000  0.0  0.0
=================== Results: RandomForestClassifier ===================
                1.0       4.0       2.0       5.0       3.0
F1         0.617674  0.455742  0.417451  0.586852  0.381261
Precision  0.568815  0.454537  0.397154  0.602632  0.444378
Recall     0.675715  0.456953  0.439935  0.571878  0.333843
